## 01 - Redes Neurais Convolucionais

In [ ]:
import tensorflow as tf
#import tensorflow.keras as tfk
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
data = pd.read_csv("data-raw/2020-Feb.csv")
data['event_time'] = pd.to_datetime(data['event_time'], format='%Y-%m-%d %H:%M:%S UTC')
data.head(2)

In [ ]:
data.shape

In [ ]:
data['date'] = data['event_time'].dt.round("1H")
pur = data[data['event_type'] == 'purchase']

g = pur.groupby(["date"], as_index=False)['price'].sum()
g = g.rename(columns={"price": "sales_value"})

g.head()

In [ ]:
g.plot(x='date', y='sales_value', figsize=(15,10))

In [ ]:
g.to_csv('data-processed/ts_hr_feb_2020.csv', index=False, header=True)

In [ ]:
data = pd.read_csv("data-processed/ts_hr_feb_2020.csv", parse_dates=['date'])
data.head()

In [ ]:
?tf.keras.layers.Conv1D

In [ ]:
numerical_i = tf.keras.layers.Input((24, 1))
numerical = tf.keras.layers.Conv1D(10, 3)(numerical_i)
numerical = tf.keras.layers.Flatten()(numerical)
dense = tf.keras.layers.Dense(200, 'relu')(numerical)
out = tf.keras.layers.Dense(24, 'relu')(dense)

mdl = tf.keras.Model(numerical_i, out)
mdl.compile('adam', 'mae')
mdl.summary()

In [ ]:
last_sunday = pd.to_datetime("2020-02-23")
train = data[data['date'] < last_sunday]
val = data[data['date'] >= last_sunday]

In [ ]:
l = 24
h = 24

X = []
Y = []
for i in range(l,train.shape[0]-h):
    f = train.iloc[i-l:i]['sales_value'].values #hoje
    y = train.iloc[i:i+h]['sales_value'].values # 1 dia depois
    
    X.append(f)
    Y.append(y)

X = np.array(X)
Y = np.array(Y)

X = np.expand_dims(X, -1)
#Y = np.expand_dims(Y, -1)

In [ ]:
mdl.fit(X, Y, batch_size=1, shuffle=True, epochs=5)

In [ ]:
Xv = []
Yv = []
for i in range(l,val.shape[0]-h):
    f = val.iloc[i-l:i]['sales_value'].values # hoje
    y = val.iloc[i:i+h]['sales_value'].values # 1 dia depois
    
    Xv.append(f)
    Yv.append(y)

Xv = np.array(Xv)
Yv = np.array(Yv)

Xv = np.expand_dims(Xv, -1)
#Y = np.expand_dims(Y, -1)

In [ ]:
p = mdl.predict(Xv)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(Yv, p)

In [ ]:
p_base = np.ones(Yv.shape)*np.median(Y, axis=0)

In [ ]:
mean_absolute_error(Yv, p_base)

In [ ]:
pd.DataFrame({'p': p[71,:], 'y': Yv[71,:]}).plot(figsize=(12,7))
plt.show()

## Fim